# Introduction to Langchain and Huggingface

Langchain: https://python.langchain.com/docs/introduction/

Openrouter for access to LLM models: https://openrouter.ai/

OpenAI API: https://platform.openai.com/api-keys

Huggingface LLM models: https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard

In [1]:
!pip install -q -U langchain langchain-openai langchain_community

In [1]:
# from google.colab import userdata
# api_key = userdata.get('API_KEY')
api_key = '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'

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='google/gemini-2.0-flash-thinking-exp:free' , temperature=0, api_key=api_key, base_url="https://openrouter.ai/api/v1")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000217B60C81D0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000217B60CA690>, root_client=<openai.OpenAI object at 0x00000217B5CC81D0>, root_async_client=<openai.AsyncOpenAI object at 0x00000217B60C8230>, model_name='google/gemini-2.0-flash-thinking-exp:free', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://openrouter.ai/api/v1')

In [3]:
llm.invoke("who are you and what can you do for me?")

AIMessage(content="Hello! I am a large language model, also known as a conversational AI or chatbot, trained to be informative and comprehensive. I am trained on a massive amount of text data, and I am able to communicate and generate human-like text in response to a wide range of prompts and questions. For example, I can provide summaries of factual topics, create stories, and translate languages.\n\nThink of me as a helpful and versatile digital assistant.  Here's a breakdown of what I can do for you:\n\n**Information and Knowledge:**\n\n* **Answer your questions:** I can access and process information from the real world through Google Search and keep my response consistent with search results.  Ask me anything you're curious about, from historical facts to scientific concepts to current events.\n* **Explain complex topics:**  I can break down complicated subjects into easier-to-understand explanations.\n* **Provide summaries:**  Need a quick overview of a long article or document? 

In [4]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [5]:
chain = prompt | llm

In [6]:
chain.invoke({"input": "how can langchain help with developing llm application?"})

AIMessage(content='Langchain is a powerful framework that significantly simplifies and accelerates the development of applications powered by Large Language Models (LLMs). It acts as a bridge, providing tools and abstractions that make it easier to interact with, orchestrate, and build complex applications on top of LLMs like GPT-3, PaLM, and others.\n\nHere\'s a breakdown of how Langchain helps with developing LLM applications, categorized by key areas:\n\n**1. Simplifying LLM Interaction and Abstraction:**\n\n* **Unified Interface for Multiple LLMs:** Langchain provides a consistent interface to interact with various LLM providers (OpenAI, Cohere, Hugging Face, etc.). You don\'t need to learn the specific API details for each provider.  This allows you to easily switch between models or even use ensembles of models without rewriting significant portions of your code.\n* **Abstraction over LLM APIs:** Langchain handles the low-level details of interacting with LLM APIs, such as authen

In [7]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [8]:
chain = prompt | llm | output_parser

In [9]:
chain.invoke({"input": "how can langchain help with developing llm application?"})

'Langchain is a powerful framework that significantly simplifies and accelerates the development of applications powered by Large Language Models (LLMs). It acts as a bridge, providing tools and abstractions that make it easier to interact with, orchestrate, and build complex applications on top of LLMs like GPT-3, PaLM, and others.\n\nHere\'s a breakdown of how Langchain helps with developing LLM applications, categorized by key areas:\n\n**1. Simplifying LLM Interaction and Abstraction:**\n\n* **Unified Interface for Multiple LLMs:** Langchain provides a consistent interface to interact with various LLM providers (OpenAI, Cohere, Hugging Face, etc.). You don\'t need to learn the specific API details for each provider.  This allows you to easily switch between models or even use ensembles of models without rewriting significant portions of your code.\n* **Abstraction over LLM APIs:** Langchain handles the low-level details of interacting with LLM APIs, such as authentication, request 

# Huggingface and Langchain

In [30]:
!pip install -q -U transformers bitsandbytes accelerate

In [10]:
import torch
import os
from langchain import PromptTemplate, HuggingFacePipeline
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline

c:\Users\Hossein\anaconda3\envs\nlp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [ ]:
# Model version of Mistral
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.1"

# Quantization is a technique used to reduce the memory and computation requirements
# of deep learning models, typically by using fewer bits, 4 bits
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Initialization of a tokenizer for the Mistral-7b model,
# necessary to preprocess text data for input
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

# Initialization of the pre-trained language Mistral-7b
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
    quantization_config=quantization_config
)

# Configuration of some generation-related settings
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
generation_config.max_new_tokens = 1024 # maximum number of new tokens that can be generated by the model
generation_config.temperature = 0.7 # randomness of the generated text
generation_config.top_p = 0.95 # diversity of the generated text
generation_config.do_sample = True # sampling during the generation process
generation_config.repetition_penalty = 1.15 # the degree to which the model should avoid repeating tokens in the generated text

# A pipeline is an object that works as an API for calling the model
# The pipeline is made of (1) the tokenizer instance, the model instance, and
# some post-procesing settings. Here, it's configured to return full-text outputs
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    generation_config=generation_config,
)

In [42]:
llm = HuggingFacePipeline(pipeline=pipe)

In [43]:
llm.invoke("who are you and what can you do for me?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'who you are and what can you do for me?\nI am an AI language model, designed to understand and respond to human language. I can assist with a wide range of tasks such as answering questions, providing information, making recommendations, and more. Is there something specific you need help with?'

In [44]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [45]:
chain = prompt | llm

In [46]:
chain.invoke({"input": "how can langchain help with developing llm application?"})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'System: You are a world class technical documentation writer.\nHuman: how can langchain help with developing llm application?\nUser 2: LangChain is an AI-powered platform that enables developers to build and deploy LLM (Language Model) applications quickly and easily. It provides pre-built models, APIs, and development tools to help users create custom LLMs for their specific use cases. With LangChain, developers can focus on building the frontend of their LLM application, while LangChain handles the backend. This allows for faster development cycles and more efficient use of resources. Additionally, LangChain offers continuous model training capabilities, which ensures that LLMs are always up-to-date and optimized for performance.'

In [47]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [48]:
chain = prompt | llm | output_parser

In [49]:
chain.invoke({"input": "how can langchain help with developing llm application?"})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"System: You are a world class technical documentation writer.\nHuman: how can langchain help with developing llm application?\n\nThe LLM (Language Model) application is an AI-powered tool that uses natural language processing (NLP) to understand and generate text in various languages. LangChain can help in developing such applications by providing a pre-built, scalable platform for NLP tasks. This platform can be used to train the LLM model on large amounts of data, allowing it to learn and generate accurate translations or summaries. Additionally, LangChain's modular architecture allows developers to easily integrate new features or custom models into the platform."